# Week 9 Assingment: What makes a good index?

## Author: Bailey Jørgensen

In [1]:
# %pip install sqlite3

# Connect to the database
import sqlite3
conn = sqlite3.connect("/courses/EDS213/big-fat.sqlite3")
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM Site")
cur.fetchone()

('barr', 'Barrow', 'Alaska, USA', 71.3, -156.6, 220.4)

### Part 1
Answer the following questions.

- Is there already an index on the Bird_nests table? If so, what is that index and will SQLite use it in the above query? Why or why not?

- Will adding an index on a column not mentioned in the WHERE clause be used by the database? Why or why not?